In [1]:
import os

In [2]:
os.chdir("../")

In [5]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class ModelEvaluationConfig:
    root_dir: Path
    X_test_data_path: Path
    y_test_data_path: Path
    model_path:Path
    scaler_file_path:Path
    metric_file_name: Path


In [6]:
from src.StockSeer.constants import *
from src.StockSeer.utils.common import read_yaml, create_directories


class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation

        create_directories([config.root_dir])

        model_evaluation_config = ModelEvaluationConfig(
            root_dir = config.root_dir,
            X_test_data_path = config.X_test_data_path,
            y_test_data_path = config.y_test_data_path,
            model_path =config.model_path,
            scaler_file_path=config.scaler_file_path,
            metric_file_name=config.metric_file_name
        )

        return model_evaluation_config

In [13]:
from tensorflow.keras.models import load_model
from joblib import load
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error, mean_squared_error
import numpy as np
from src.StockSeer.utils.common import save_json
from src.StockSeer.logging import logger

class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config

    def eval_metrics(self,actual, pred):

        mape = mean_absolute_percentage_error(actual, pred) * 100
        rmse = np.sqrt(mean_squared_error(actual, pred))
        mae = mean_absolute_error(actual, pred)

        return mape, rmse, mae
    
    def save_results(self):
        X_test = np.load(self.config.X_test_data_path)
        y_test = np.load(self.config.y_test_data_path)
        model = load_model(self.config.model_path)
        scaler = load(self.config.scaler_file_path)
        
        predictions = model.predict(X_test)
        predictions = scaler.inverse_transform(predictions)
        print(predictions.shape)
        (mape, rmse, mae) = self.eval_metrics(y_test,predictions)

        # Saving metrics as local
        scores = {"mape(%)": mape, "rmse": rmse, "mae": mae}
        save_json(path=Path(self.config.metric_file_name), data=scores)


In [14]:
try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation_config = ModelEvaluation(config=model_evaluation_config)
    model_evaluation_config.save_results()
except Exception as e:
    raise e

[2024-03-05 15:13:28,442: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-03-05 15:13:28,442: INFO: common: yaml file: params.yaml loaded successfully]
[2024-03-05 15:13:28,442: INFO: common: yaml file: schema.yaml loaded successfully]
[2024-03-05 15:13:28,450: INFO: common: created directory at: artifacts]
[2024-03-05 15:13:28,451: INFO: common: created directory at: artifacts/model_evaluation]
7/7 [==============================] - 2s 29ms/step
(200, 1)
[2024-03-05 15:13:31,039: INFO: common: json file saved at: artifacts\model_evaluation\metrics.json]
[2024-03-05 15:13:31,039: INFO: 2481893614: model evalution metric stored at artifacts/model_evaluation/metrics.json]
